In [1]:



ъimport scipy.optimize
import numpy as np
import math

In [2]:
c = [2, 3, 0, -1, 0, 0]
c_ = [-2, -3, 0, 1, 0, 0]
A = [[2, -1, 0, -2, 1, 0],
    [3, 2, 1, -3, 0, 0],
    [-1, 3, 0, 4, 0, 1]]
b = [16, 18, 24]
simplex_steps = []

In [3]:
scipy.optimize.linprog(c_,
                       A_eq=A,
                       b_eq=b, 
                       method='simplex',
                      callback = lambda xk, **kwargs: simplex_steps.append([xk.copy(), kwargs.copy()])
                      )

C:\Users\Admin\AppData\Local\Temp\ipykernel_17584\3456479888.py:1: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  scipy.optimize.linprog(c_,


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -25.636363636363637
       x: [ 5.455e-01  8.182e+00  0.000e+00  0.000e+00  2.309e+01
            0.000e+00]
     nit: 4

### Самостоятельная реализация симплекс-метода

In [4]:
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)


def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]


def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])


def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)

    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column


def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]

    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value

    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier

    return new_tableau


def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1


def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)

    return solutions


solution = simplex(c, A, b)
print('solution: ', solution)

solution:  [0.5454545454545459, 8.181818181818182, 0, 0, 23.090909090909086, 0]


**Результат сошелся с решением на бумаге.**